Representations:
* Individual tree
* Trajectory
    * Radii vs. iteration, with summaries such as log-probability, and the marginal likelihoods of various models
    * Include SMIRKS tags for the individual types

In [2]:
from pickle import load

In [3]:
with open('../bayes_implicit_solvent/rjmc_experiments/tree_rjmc_run_64.pkl', 'rb') as f:
    result = load(f)

using the following decorators
